# Collusive Voting

## Introduction

It is a well-documented phenomenon that Eurovision members generally have voting biases towards (and against) other specific members, suggesting that countries may not be voting entirely on the quality and popularity of entry songs alone. 

To this end, we implement a modified version of the collusion detection algorithm introduced in [Derek Gatherer's paper in the Journal of Artificial Societies and Social Simulation](http://jasss.soc.surrey.ac.uk/9/2/1.html). 


## 1. Reading and cleaning data

The data is obtained from [datagraver](https://data.world/datagraver/eurovision-song-contest-scores-1975-2019/workspace/file?filename=eurovision_song_contest_1975_2019v5.xlsx), with some incorrect information edited out, such as Belarus giving Russia both 12 (correct) and 0 (incorrect) televote points in the 2019 grand final.

### Import modules

In [ ]:
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import networkx as nx


import itertools

# initialize random seed
from numpy.random import default_rng
rng = default_rng()


# import common functions
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/modules")

import functions as common_functions


### Read data

In [ ]:
# original:https://data.world/datagraver/eurovision-song-contest-scores-1975-2019/workspace/file?filename=eurovision_song_contest_1975_2019v5.xlsx
# note that the data for 2019 Final: Belarus has been corrected by removing incorrect score reports. In addition, 2018f's "The Netherands" has been corrected to "The Netherlands".
csv = pd.read_csv("./eurovision_song_contest_1975_2019v5.csv")

csv.head()


### Clean / modify CSV data

In [ ]:
# rename columns
csv = csv.rename(columns={"Jury or Televoting" : "Jury_or_Televoting", "From country": "From_country", "To country" : "To_country"})

# select only grand finals data
csv = csv[ (csv["(semi-) final"] == "f") & (csv["Duplicate"] != "x") & (csv["From_country"] != csv["To_country"]) ] 

# select only relevant columns and drop duplicates
csv = csv[ ["Year", "Jury_or_Televoting", "From_country", "To_country", "Points"] ].drop_duplicates()

# rename FYR Macedonia to North Macedonia. This choice is arbitrary and not meant to be political.
csv["From_country"].replace({"F.Y.R. Macedonia": "North Macedonia"}, inplace=True)
csv["To_country"].replace({"F.Y.R. Macedonia": "North Macedonia"}, inplace=True)

csv.head()


In [ ]:
# for collusive voting, combine the jury and televoting votes.
combined_csv = csv.join(
    csv.set_index(["Year", "From_country", "To_country"]), 
    on=["Year", "From_country", "To_country"], 
    lsuffix="_Jury", 
    rsuffix="_Televote",
    how="right"
)
combined_csv = combined_csv[(combined_csv.Year < 2016) | ( (combined_csv.Year >= 2016) & (combined_csv.Jury_or_Televoting_Jury == "J") & (combined_csv.Jury_or_Televoting_Televote == "T") )]

combined_csv.loc[combined_csv.Year <= 2015, "Points_Televote"] = 0 # Dual voting begun in 2016

combined_csv.drop(['Jury_or_Televoting_Jury', 'Jury_or_Televoting_Televote'], axis=1, inplace=True)

combined_csv["Points_total"] = combined_csv["Points_Jury"] + combined_csv["Points_Televote"]

# sample
combined_csv[  (combined_csv.From_country == "Belgium") & (combined_csv.Year == 2019)].sort_values(["Points_Televote", "Points_Jury"], ascending=False).head()

In [ ]:
def max_each_year(csv):
    grouped = csv[ ["Year", "From_country", "Points_Jury", "Points_Televote"] ].groupby(["Year", "From_country"]).max()
    by_year = grouped.groupby(["Year"]) 
    max_possible_points = by_year.Points_Jury.max() + by_year.Points_Televote.max()
    televote_by_year = (by_year.Points_Televote.max() != 0)
    return max_possible_points, televote_by_year

# calculate the maximum amount of points a country can receive each year, and put it into a column. Note that this not simply "number of countries voting * 12". 
MAX_POSSIBLE_POINTS_RECEIVED, HAS_TELEVOTE_BY_YEAR = max_each_year(combined_csv)
points = combined_csv.join(MAX_POSSIBLE_POINTS_RECEIVED.to_frame(name="highest"), on=["Year"])
points.Points_Jury /= points["highest"]
points.Points_Televote /= points["highest"]
points.Points_total /= points["highest"]

points.head()

In [ ]:
PARTICIPANTS_BY_YEAR = points.groupby(["Year", "From_country"])
PARTICIPANTS_BY_YEAR = PARTICIPANTS_BY_YEAR.count()["To_country"].groupby(level="Year").count()
PARTICIPANTS_BY_YEAR.tail()

## 2. Favouritism and Collusion Detection Algorithm

### Algorithm

Question: Did country X give an abnormal amount of points (indicating favouritism) from years A to B?

Define `FAVOURITISM(X, Y, A, B)` as follows:

0. Determine which years that X actually voted for Y (since it is possible that X could not vote for Y in some years). Let this list of years be `TIMESPAN`.

1. Obtain the number of points given from X to Y in the `TIMESPAN`, divided by the maximum number of points a country can receive each year. Take the average of those values (which are percentages) be $\bar{x}$.
	* Using a percentage of the maximum number of points received allows us to take into accounts of years with separate televotes.

2. Simulate many scenarios where country X gives country Y a random number of points, averaged over `TIMESPAN`. Collect these simulations into a list called $S$.

3. Sort $S$ to obtain a simulated average vote which follows a normal distribution.

4. If $\bar{x}$ is greater than the 5th _highest_ percentile of $S$, then we can say with 95% confidence that country X has collusively voted for country Y.
	* In other words, country X has given country Y an exceptionally and abnormally high amount of points, indicating favouritism.

5. If the same is true for `FAVOURITISM(Y, X, A, B)` then there is collusion between countries X and Y.
	* That is, `COLLUSION(X, Y, A, B) == (FAVOURITISM(X, Y, A, B) and FAVOURITISM(Y, X, A, B))`.

The full algorithm is shown below:

```
INPUT: START_YEAR, END_YEAR, SIGNIFICANCE, TABULATION (of actual scores)
For each pair of countries (DONOR to RECIPIENT) {
	From TABULATION, select actual results DONOR to RECIPIENT between START_YEAR and END_YEAR
	Calculate actual average vote from DONOR to RECIPIENT between START_YEAR and END_YEAR
	Initialise array AVERAGE_SIMULATION
	For 100000 iterations {
		# simulate an average score that DONOR gives to RECIPIENT from START_YEAR to END_YEAR
		Initialise array ONE_SIMULATION 
		For each year from START_YEAR to END_YEAR { 
			Determine NUM, number of countries voting that year
			Derive simulated position by RAND * (NUM-1), rounded up to nearest integer
			According to simulated position, award simulated vote	
			Add simulated vote to array ONE_SIMULATION
		}
		Determine average simulated vote as average of array ONE_SIMULATION
		Add simulated vote to array AVERAGE_SIMULATION
	}
	Sort AVERAGE_SIMULATION and determine the vote at the top 5th percentile
	If actual average vote is greater than the 5th percentile of AVERAGE_SIMULATION
		Then DONOR votes significantly for RECIPIENT at the 5% level
}

If the same is true for RECIPIENT to DONOR, then there is collusive voting at the 5% significance level.

```

Auxillary function to simulate jury and public votes.

In [ ]:
## vote simulation

def simulate_jury_votes(countries_each_year, num_simulations:int = 10):
    """
    Note that with numpy we can create the simulation without running ANY for loops.
    To do this, randomize the ranks that country X gives to country Y.
    Each row is a simulation over the TIMESPAN. 
        - In other words, one simulation of country X randomly giving its points to Y over the course of chosen years.
    Each column is the ranking awarded to country Y that year.
    """
    ranks = pd.DataFrame(
        data = rng.integers(low=1, high=countries_each_year, size=(num_simulations,len(countries_each_year))),
        columns = countries_each_year.index
    )

    # to verify if the randomized ranks are correct: 
    #   set run ranks.max() many times and make sure it does not exceed PARTICIPANTS_BY_YEAR
    assert (countries_each_year - ranks.max()).sum() >= 0

    # convert ranks to points (the np.where with np.searchsorted thing), then convert to %
    points = np.where(ranks > 10, 0, 
        12 - np.searchsorted([1,1.5,2,2.5,3,4,5,6,7,8,9,10], ranks)
    )
    return points

def simulate_televotes(countries_each_year, no_televote, num_simulations : int = 10):
    """
    this more or less the same as the jury voting, except years with no televotes are set to 0.
    """
    ranks = pd.DataFrame(
        data = rng.integers(low=1, high=countries_each_year, size=(num_simulations,len(countries_each_year))),
        columns = countries_each_year.index
    )
    assert (countries_each_year - ranks.max()).sum() >= 0

    ranks[no_televote.index] = 256 # arbitrary, this is just some value that is more that 10, so it will be set to 0 later
    # convert ranks to points (the np.where with np.searchsorted thing), then convert to %
    points = np.where(ranks > 10, 0, 
        12 - np.searchsorted([1,1.5,2,2.5,3,4,5,6,7,8,9,10], ranks)
    )
    return points


Auxillary function to graph simulated voting.

In [ ]:
    
def graph_distribution(simluated_distribution, actual_mean, **plot_data):
    S = pd.Series(simluated_distribution)

    # plot density graph
    plt.hist(S, color = 'blue', edgecolor = 'black', bins = 12)
    # draw actual mean
    plt.axvline(x=actual_mean, label=f'actual mean ({"{0:.2f}".format(actual_mean)})', c='r')
    # ax = S.plot.kde()

    # other information used for plotting graph
    from_country, to_country = plot_data.get("From", ""), plot_data.get("To", "")
    from_year, to_year = plot_data.get("start", ""), plot_data.get("end", "")
    num_simulations = plot_data.get("n_sim", "")
    has_favouritism = plot_data.get("has_favouritism", False)


    title = f"""
    Voting simulation
    { '{} to {}'.format(from_country, to_country) if (from_country and to_country) else '' } {'(favouritism detected)' if has_favouritism else ''}
    {'({} to {})'.format(from_year, to_year) if (from_year and to_year) else '' }
    {'(n = {} rounds)'.format(num_simulations) if num_simulations else ''}
    """

    # setting up title
    plt.title(title)
    plt.ylabel("frequency")
    plt.xlabel("average % points given (out of max. possible points given)")

    # show legend (for actual_mean) and plot
    plt.legend()
    plt.show()



Define favouritism function.

In [ ]:
def favouritism(
    csv, 
    country_x: str, country_y: str,
    start_year: int, end_year: int,
    num_simulations:int=100000,
    confidence_threshold=95,
    plot_data=False,
    return_detailed_data=False,
):

    # countries cannot vote for themselves
    if country_x == country_y: return False

    # get relevant years and columns
    by_year = csv[
        (start_year <= csv.Year) & (csv.Year <= end_year) 
        & (csv.From_country == country_x) & (csv.To_country == country_y) 
    ]

    if len(by_year) == 0: 
        # print(f"could not find records for {country_x} -> {country_y} (make sure the spelling is correct)")
        return None

    # Determine which years that X actually voted for Y (since it is possible that X could not vote for Y in some years)
    timespan = by_year.Year

    # determine number of countries each year
    countries_each_year = PARTICIPANTS_BY_YEAR[ timespan ]

    # determine $\bar{x}$, the average amount of points (as a %) that X gave to Y over the course of the timespan.
    actual_average = by_year.Points_total.mean()

    # determine which years did not have any televotes
    has_televote = HAS_TELEVOTE_BY_YEAR[ timespan ]
    no_televote = has_televote[has_televote == False]

    # simulate jury and public votes. for televotes, years that had no televotes just return 0.
    jury_votes, televotes = simulate_jury_votes(countries_each_year, num_simulations), simulate_televotes(countries_each_year, no_televote, num_simulations)
    simulated_points = jury_votes + televotes
    points_percentage = simulated_points / MAX_POSSIBLE_POINTS_RECEIVED[timespan].values # normalisation

    # generate S from simulated points
    simulation_avgs = points_percentage.mean(axis=1)
    # sort S to obtain a simulation of a normal distribution
    simluated_distribution = np.sort(simulation_avgs, axis=None)
    # sanity check
    assert simluated_distribution.min() >= 0

    # determine if favouritism occurred
    simulated_95th_percentile = np.percentile(simluated_distribution, confidence_threshold)
    has_favouritism = actual_average > simulated_95th_percentile
    # if has_favouritism: print(f"Favouritism from {country_x} to {country_y}")

    # plot data
    if plot_data:
        graph_distribution(
            simluated_distribution, actual_average, 
            n_sim=num_simulations,
            From=country_x, To=country_y,
            start=start_year, end=end_year,
            has_favouritism=has_favouritism
        )

    if return_detailed_data:
        if has_favouritism:
            return {
                "actual_avg": actual_average,
                "sim_avg": simulated_95th_percentile,
            }
        else:
            return False
    else:
        return has_favouritism



# verify searchsorted is working as intended (not required)
# assert (
#     ( 12 - np.searchsorted([1,1.5,2,2.5,3,4,5,6,7,8,9,10], [1,2,3,4,5,6,7,8,9,10]) ) 
#     ==
#     np.array([12,10,8,7,6,5,4,3,2,1])
# ).all() == True

Define collusion function.

In [ ]:
def collusion(
    csv,
    country_x: str, country_y: str,
    start_year: int, end_year: int,
    **kwargs
):
    return favouritism(csv, country_x, country_y, start_year, end_year, **kwargs) and \
           favouritism(csv, country_y, country_x, start_year, end_year, **kwargs)
    


Auxillary function to collect detection results.

In [ ]:
## matrix to store favouritism/collusion
def get_favouritism_matrix(points, start_year: int = 1999, end_year: int = 2019):
    filtered_by_year = points[(start_year <= points.Year) & (points.Year <= end_year)]
    all_participants = sorted(set(filtered_by_year["From_country"].unique()) | set(filtered_by_year["To_country"].unique()))
    favouritism_matrix = pd.DataFrame(
        index=all_participants,
        columns=all_participants,
        data=False
    )
    for donor in favouritism_matrix.index:
        for recipient in favouritism_matrix.columns:
            result = favouritism(points, donor, recipient, start_year, end_year)
            if result == None:
                pass
                # print(f"no points were given from {donor} to {recipient} ({start_year}-{end_year})")
            if result == True:
                favouritism_matrix.at[donor, recipient] = True
    return favouritism_matrix



## 3. Run detection algorithm and collect results

To this end, we can use several of the functions defined above to determine the favouritism under various circumstances. 


### Determine favouritism from country A to country B individually

To determine whether country A voted for country B an exceptional amount of times (indicating favouritism) over a span of several years, simply run the `favouritism` function.

In [ ]:
## check favouritism individually

# determine whether Norway showed favouritism to Sweden from 2000 to 2018
print(favouritism(points, "Norway", "Sweden", 2000, 2018))
# determine whether Sweden showed favouritism to Norway from 2000 to 2018
print(favouritism(points, "Sweden", "Norway", 2000, 2018))
# determine whether Norway and Sweden voted collusively (showed favourtism towards each other) from 2000 to 2018
print(collusion(points, "Sweden", "Norway", 2000, 2018))

# determine whether Croatia showed favouritism to Germany from 2014 to 2019
print(favouritism(points, "Croatia", "Germany", 2014, 2019))

# determine whether Malta and Cyprus voted collusively (showed favourtism towards each other) from 2000 to 2018
print(collusion(points, "Malta", "Cyprus", 2000, 2018))


### Determine favouritism between all countries

For this, run `get_favouritism_matrix` and select a specific timeframe of years to determine favouritism on, e.g. from 2015 to 2019.

Note that this function will take some time to finish.

The output of this function is a 2D matrix, where the position (A, B) is the favouritism from A to B. 

For example, if `favouritism_matrix["Belgium", "Bulgaria"]` is `True`, then Belgium showed favouritism to Bulgaria within the specified timeframe.

In [ ]:
favouritism_matrix = get_favouritism_matrix(points, 2015, 2019)

favouritism_matrix

From `get_favouritism_matrix`, we can quickly determine which countries a certain country shows favouritism to.

In [ ]:
list(favouritism_matrix.loc["Cyprus"][favouritism_matrix.loc["Cyprus"] == True].index)

## 4. Visualise results

Visualise the results of favouritism from `get_favouritism_matrix`.

Note that for large amounts of countries, this map becomes very hard to read.

In [ ]:
def draw_favouritism_graph_from_adjacency_matrix(matrix):
    rows, cols = np.where(matrix)
    edges = list(zip(matrix.index[rows], matrix.columns[cols]))

    G = nx.DiGraph()
    G.add_edges_from(edges)
    positions = nx.circular_layout(G, scale=0.9, center=(0,0))

    def label_pos(x, y, move):
        return (x*move, y*move)

    label_positions = {n: label_pos(x, y, 1.15)
                 for n, (x, y) in positions.items()}
    nx.draw_networkx(G, pos=positions, with_labels=False, node_size=40)
    nx.draw_networkx_labels(G, pos=label_positions, font_size=10, font_color="green")

draw_favouritism_graph_from_adjacency_matrix(favouritism_matrix)


Visualise results of countries that collude with each other.

In [ ]:
def draw_collusion_graph_from_adjacency_matrix(matrix):
    collusion_matrix = matrix & matrix.transpose()
    draw_favouritism_graph_from_adjacency_matrix(collusion_matrix)

draw_collusion_graph_from_adjacency_matrix(favouritism_matrix)


## 5. Save favouritism results into a dataframe.

This is an alternative to producing an entire favouritism matrix, which is usually very sparse.

Instead, we can save just the instances of favouritism into a dataframe further analysis.

In [ ]:
def compile_one_result(start_year, end_year, num_simulations=100000, confidence_threshold=95):

    results = []

    filtered_by_year = points[(start_year <= points.Year) & (points.Year <= end_year)]
    all_participants = sorted(set(filtered_by_year["From_country"].unique()) | set(filtered_by_year["To_country"].unique()))
    for donor in all_participants:
        for recipient in all_participants:
            result = favouritism(points, donor, recipient, start_year, end_year,
                num_simulations=num_simulations,
                confidence_threshold=confidence_threshold,
                return_detailed_data=True
            )
            if result:
                results += [(start_year, end_year, donor, recipient, result["actual_avg"], result["sim_avg"])]
    return results

def compile_favouritism_results(years_list, num_simulations=100000, confidence_threshold=95):
    columns = ("start_year", "end_year", "from_country", "to_country", "real_avg", "sim_avg")
    results = []
    for years in years_list:
        result = compile_one_result(
            years[0], years[1],
            num_simulations, confidence_threshold
        )
        if result: results += result
    
    result_dataframe = pd.DataFrame(results, columns=columns)
    params = {
        "num_simulations": [num_simulations],
        "confidence_threshold": [confidence_threshold],
    }
    parameters = pd.DataFrame(data=params)
    return result_dataframe, parameters


To use `compile_results`, supply it with a list of time intervals.

In [ ]:
years = [
    (1975, 1980),
    (1981, 1985),
    # (1986, 1990),
    # (1991, 1995),
    # (1996, 2000),
    # (2001, 2005),
    # (2006, 2010),
    # (2011, 2015),
    # (2016, 2019),
]

result_dataframe, parameters = compile_favouritism_results(years, 100000)
result_dataframe
